# Finetuning Hugging Face Wav2Vec2 model on LibriSpeech dataset 

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
%cd drive/MyDrive

/content/drive/MyDrive


In [3]:
!pip install huggingface_hub
!pip install datasets>=1.18.3
!pip install transformers==4.11.3
!pip install librosa
!pip install jiwer

     |████████████████████████████████| 67 kB 3.2 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
     |████████████████████████████████| 2.9 MB 5.1 MB/s 
     |████████████████████████████████| 895 kB 45.6 MB/s 
     |████████████████████████████████| 3.3 MB 35.7 MB/s 
     |████████████████████████████████| 596 kB 44.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 50 kB 3.3 MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149861 sha256=e0e4068e11b9cd43ef018c94c1a17d3aca4d6adb93293db246ad4250507a418b
  Stored in directory: /root/.cache/pip/wheels/05/5f/c

In [4]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-470
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  git-lfs
0 upgraded, 1 newly installed, 0 to remove and 39 not upgraded.
Need to get 2,129 kB of archives.
After this operation, 7,662 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 git-lfs amd64 2.3.4-1 [2,129 kB]
Fetched 2,129 kB in 1s (1,926 kB/s)
Selecting previously unselected package git-lfs.
(Reading database ... 155320 files and directories currently installed.)
Preparing to unpack .../git-lfs_2.3.4-1_amd64.deb ...
Unpacking git-lfs (2.3.4-1) ...
Setting up git-lfs (2.3.4-1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...


In [5]:
!git config --global credential.helper store

## Init HuggingFace hub

In [6]:
from huggingface_hub import notebook_login

In [7]:
notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token


In [8]:
repo_name = "wav2vec2-finetuning-model"

## LibriSpeech dataset

In [9]:
import numpy as np
import torch
import torchaudio
from datasets import Dataset, DatasetDict, load_dataset

In [6]:
dataset = {
    "train": torchaudio.datasets.LIBRISPEECH(root="", url="dev-clean"), 
    "test": torchaudio.datasets.LIBRISPEECH(root="", url="test-clean")
}
'''dataset = {
    "train": torchaudio.datasets.LIBRISPEECH(root="", url="train-clean-100"), 
    "val": torchaudio.datasets.LIBRISPEECH(root="", url="dev-clean"), 
    "test": torchaudio.datasets.LIBRISPEECH(root="", url="test-clean")
}'''

'dataset = {\n    "train": torchaudio.datasets.LIBRISPEECH(root="", url="train-clean-100"), \n    "val": torchaudio.datasets.LIBRISPEECH(root="", url="dev-clean"), \n    "test": torchaudio.datasets.LIBRISPEECH(root="", url="test-clean")\n}'

In [10]:
from collections import defaultdict
from tqdm import tqdm
import pandas as pd

## Push data to HuggingFace Hub

In [11]:
def to_datasetdict(torch_dataset):
    data = defaultdict(list)
    for idx, d in enumerate(tqdm(torch_dataset)):
        data['audio'].append(d[0][0])
        data['sr'].append(d[1])
        data['text'].append(d[2].lower())
    
    return Dataset.from_dict(dict(data))

In [36]:
dataset['train'] = to_datasetdict(dataset['train'])
dataset['test'] = to_datasetdict(dataset['test'])

100%|██████████| 2620/2620 [11:14<00:00,  3.89it/s]


In [54]:
data = DatasetDict(dataset)

In [43]:
dataset_name = "librispeech"

In [ ]:
data.push_to_hub(dataset_name, private=True)

## Load data

In [12]:
dataset = load_dataset('timkakhanovich/librispeech', use_auth_token=True)

Downloading:   0%|          | 0.00/848 [00:00<?, ?B/s]

Using custom data configuration timkakhanovich--librispeech-92a89147dcaf484d


  0%|          | 0/2 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/200M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/224M [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/206M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/207M [00:00<?, ?B/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/parquet/timkakhanovich--librispeech-92a89147dcaf484d/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
def transform_dataset(sample):
    audio = sample[0][0]
    sampling_rate = sample[1]
    text = sample[2].lower()
    
    return {
        "audio": sample[0][0], 
        "sampling_rate": sample[1], 
        "text": text
    }

In [10]:
for split in dataset.keys():
    dataset[split] = dataset[split].map(transform_dataset)

## Vocab

In [13]:
import string
import json

In [14]:
vocab = {w: idx for idx, w in enumerate(string.ascii_lowercase)}

In [15]:
vocab.update({
    "|": len(vocab), 
    "'": len(vocab) + 1, 
    "<UNK>": len(vocab) + 2, 
    "<PAD>": len(vocab) + 3
})

In [16]:
vocab

{"'": 27,
 '<PAD>': 29,
 '<UNK>': 28,
 'a': 0,
 'b': 1,
 'c': 2,
 'd': 3,
 'e': 4,
 'f': 5,
 'g': 6,
 'h': 7,
 'i': 8,
 'j': 9,
 'k': 10,
 'l': 11,
 'm': 12,
 'n': 13,
 'o': 14,
 'p': 15,
 'q': 16,
 'r': 17,
 's': 18,
 't': 19,
 'u': 20,
 'v': 21,
 'w': 22,
 'x': 23,
 'y': 24,
 'z': 25,
 '|': 26}

In [17]:
with open(r'vocab.json', 'w') as vocab_file:
    json.dump(vocab, vocab_file)

## Text tokenizer

In [18]:
from transformers import Wav2Vec2CTCTokenizer

In [19]:
tokenizer = Wav2Vec2CTCTokenizer(
    "./vocab.json", unk_token="<UNK>", pad_token="<PAD>", word_delimiter_token="|"
)

In [20]:
tokenizer.push_to_hub(repo_name)

## Audio Wav2Vec2 processing

In [21]:
from transformers import (
    Wav2Vec2FeatureExtractor, 
    Wav2Vec2Processor
)

In [22]:
feature_extractor = Wav2Vec2FeatureExtractor(
    feature_size=1, sampling_rate=16000, padding_value=0.0, 
    do_normalize=True, return_attntion_mask=False
)

In [23]:
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

## Preprocessing dataset

In [24]:
def data_preprocessing(sample):
    sample['input_values'] = processor(sample['audio'], sampling_rate=sample["sr"]).input_values[0]
    
    with processor.as_target_processor():
        sample['labels'] = processor(sample['text']).input_ids
    
    return sample

In [28]:
for split in dataset.keys():
    dataset[split] = dataset[split].map(data_preprocessing)

0ex [00:00, ?ex/s]

0ex [00:00, ?ex/s]

## Data Collator

In [29]:
import torch

from dataclasses import dataclass, field
from typing import Optional, Union

In [30]:
@dataclass
class DataCollatorCTCWithPadding:

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, samples):
        input_features = [{'input_values': s['input_values']} for s in samples]
        label_features = [{'input_ids': s['labels']} for s in samples]

        batch = self.processor.pad(
            input_features, 
            padding=self.padding, 
            max_length=self.max_length, 
            pad_to_multiple_of=self.pad_to_multiple_of, 
            return_tensors="pt"
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features, 
                padding=self.padding, 
                max_length=self.max_length_labels, 
                pad_to_multiple_of=self.pad_to_multiple_of_labels, 
                return_tensors="pt"
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch['input_ids'].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch['labels'] = labels
        
        return batch

In [31]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

## Metrics

In [32]:
from datasets import load_metric

In [33]:
wer_metric = load_metric('wer')

Downloading:   0%|          | 0.00/1.90k [00:00<?, ?B/s]

In [34]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)
    
    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id
    
    pred_str = processor.batch_decode(pred_ids)
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)
    
    wer = wer_metric.compute(predictions=pred_str, references=label_str)
    
    return {'wer': wer}

## Model

In [35]:
from transformers import Wav2Vec2ForCTC

In [36]:
model = Wav2Vec2ForCTC.from_pretrained(
    'facebook/wav2vec2-base', 
    ctc_loss_reduction='mean', 
    pad_token_id=processor.tokenizer.pad_token_id
)

Downloading:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py:337: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "


Downloading:   0%|          | 0.00/363M [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/wav2vec2-base were not used when initializing Wav2Vec2ForCTC: ['project_q.weight', 'quantizer.weight_proj.bias', 'project_hid.bias', 'project_q.bias', 'project_hid.weight', 'quantizer.weight_proj.weight', 'quantizer.codevectors']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['lm_head.bias', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predicti

In [37]:
model.freeze_feature_extractor()

## Init arguments

In [38]:
from transformers import TrainingArguments

In [39]:
training_args = TrainingArguments(
    output_dir=repo_name, 
    group_by_length=True, 
    per_device_train_batch_size=32, 
    evaluation_strategy='steps', 
    num_train_epochs=30, 
    fp16=True, 
    gradient_checkpointing=True, 
    save_steps=500, 
    eval_steps=500, 
    logging_steps=500, 
    learning_rate=1e-4, 
    weight_decay=5e-3, 
    warmup_steps=1000, 
    save_total_limit=2
)

## Training...

In [40]:
from transformers import Trainer

In [41]:
trainer = Trainer(
    model=model, 
    data_collator=data_collator, 
    args=training_args, 
    compute_metrics=compute_metrics, 
    train_dataset=dataset['train'], 
    eval_dataset=dataset['test'], 
    tokenizer=processor.feature_extractor
)

Using amp fp16 backend


In [43]:
trainer.train()

In [ ]:
trainer.push_to_hub()